## Connecting to database

In [ ]:
import psycopg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv

load_dotenv()

netid = os.getenv('netid')
password = os.getenv('password')

API_KEY = os.getenv('TMDBAPIKEY')

# define the connection parameters
dbname = 'everything2024'
user = netid
password = password
host = 'pg.mlds.northwestern.edu'
port = '5432' 

79001bd18f242ee506f2bb4bc3cb8ce0


In [19]:
connection = psycopg.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

In [20]:
cursor = connection.cursor()
cursor.execute(
    """
    select * from mlds422.movies;

    """
)

result = cursor.fetchall()


In [54]:
movies_df = pd.DataFrame(result, columns = [column[0] for column in cursor.description])

In [ ]:
## Svae data to pkl file

# movies_df.to_pickle("movies_df_original.pkl")

# print("DataFrame saved to 'movies_df.pkl'")

# movies_df = pd.read_pickle("movies_df.pkl")

## EDA

In [57]:
display(movies_df.head())
movies_df.info()

,movie_id,movie_title,movie_info,rating,genre,directors,in_theaters_date,on_streaming_date,runtime_in_minutes,critic_rating,critic_count,audience_rating,audience_count
0,1,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,2010-02-12,2010-06-29,83.0,49,144,53.0,254287.0
1,2,Please Give,Kate has a lot on her mind. There's the ethics...,R,Comedy,Nicole Holofcener,2010-04-30,2010-10-19,90.0,86,140,64.0,11567.0
2,3,10,Blake Edwards' 10 stars Dudley Moore as George...,R,"Comedy, Romance",Blake Edwards,1979-10-05,1997-08-27,118.0,68,22,53.0,14670.0
3,4,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",None,"Classics, Drama",Sidney Lumet,2057-04-13,2001-03-06,95.0,100,51,97.0,105000.0
4,5,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,2054-01-01,2003-05-20,127.0,89,27,74.0,68860.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16638 entries, 0 to 16637
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            16638 non-null  int64  
 1   movie_title         16638 non-null  object 
 2   movie_info          16614 non-null  object 
 3   rating              11776 non-null  object 
 4   genre               16621 non-null  object 
 5   directors           16524 non-null  object 
 6   in_theaters_date    15823 non-null  object 
 7   on_streaming_date   16636 non-null  object 
 8   runtime_in_minutes  16483 non-null  float64
 9   critic_rating       16638 non-null  int64  
 10  critic_count        16638 non-null  int64  
 11  audience_rating     16381 non-null  float64
 12  audience_count      16386 non-null  float64
dtypes: float64(3), int64(3), object(7)
memory usage: 1.7+ MB


### 1. Find the number of movies that were released in theatres each year.

In [ ]:
# Convert in_theaters_date to datetime
movies_df['in_theaters_date'] = pd.to_datetime(movies_df['in_theaters_date'])

# Chekck erroneous values
display(movies_df['in_theaters_date'].min())
display(movies_df['in_theaters_date'].max())
display(movies_df[movies_df['in_theaters_date'].dt.year > 2024])

# They are wrong in century, they should be 20th century not 21st
# Fix by -100 in year
movies_df['in_theaters_date'] = movies_df['in_theaters_date'].apply(lambda x : x.replace(year=x.year-100) if x.year >2024 else x)

# Check if erroneous values are fixed
display(movies_df[movies_df['in_theaters_date'].dt.year > 2024])

Timestamp('1970-01-01 00:00:00')

Timestamp('2069-12-31 00:00:00')

,movie_id,movie_title,movie_info,rating,genre,directors,in_theaters_date,on_streaming_date,runtime_in_minutes,critic_rating,critic_count,audience_rating,audience_count
3,4,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",None,"Classics, Drama",Sidney Lumet,2057-04-13,2001-03-06,95.0,100,51,97.0,105000.0
4,5,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,2054-01-01,2003-05-20,127.0,89,27,74.0,68860.0
6,7,The 39 Steps,A man in London tries to help a counterespiona...,None,"Action & Adventure, Classics, Mystery & Suspense",Alfred Hitchcock,2035-08-01,2035-06-06,87.0,96,47,86.0,23827.0
7,8,3:10 to Yuma,"Desperate for money, frontier rancher Van Hefl...",None,"Classics, Drama, Western",Delmer Daves,2057-08-07,2002-04-02,92.0,96,27,78.0,9224.0
9,10,Abraham Lincoln,"To date, this D.W. Griffith epic is the only t...",None,"Classics, Drama",D.W. Griffith,2030-11-08,2008-11-18,97.0,90,10,38.0,455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16572,16572,Young Mr. Lincoln,"In this biography of Abraham Lincoln, the Pres...",None,"Classics, Drama",John Ford,2039-06-09,2006-02-14,100.0,100,22,81.0,2298.0
16574,16574,The Young Savages,The Young Savages is what used to be called a ...,None,"Classics, Drama",John Frankenheimer,2061-05-24,2014-09-23,110.0,50,6,47.0,152.0
16600,16600,Z,"Based on true events, 'Z' is the award-winning...",PG,"Art House & International, Drama, Mystery & Su...",Costa-Gavras,2069-02-26,2002-07-02,128.0,93,42,93.0,6506.0
16635,16636,Zorba the Greek,If ever there was a role that Anthony Quinn wa...,None,"Action & Adventure, Art House & International,...",None,2064-12-17,2004-08-03,142.0,78,9,87.0,7126.0


,movie_id,movie_title,movie_info,rating,genre,directors,in_theaters_date,on_streaming_date,runtime_in_minutes,critic_rating,critic_count,audience_rating,audience_count


In [68]:
# 815 null values in in_theaters_date
movies_df['in_theaters_date'].isna().sum()

# Missing value tittles
moview_titles = movies_df[movies_df['in_theaters_date'].isna()]['movie_title']
moview_titles


61       All Quiet on the Western Front
75                          The Silence
161                          The Broken
271                    Three Blind Mice
299                      Avenging Angel
                      ...              
16553    You Got Served: Beat The World
16556           You Might Be the Killer
16585                Your Son (Tu Hijo)
16602                          Zambezia
16619                  Ziegfeld Follies
Name: movie_title, Length: 815, dtype: object

In [ ]:
# Using TMDB API to map missing values in in_theaters_date
import requests
API_KEY = os.getenv('TMDBAPIKEY')
url = "https://api.themoviedb.org/3/search/movie"
print(API_KEY)

79001bd18f242ee506f2bb4bc3cb8ce0


In [91]:
def search_movie(movie_title):
    """Search for a movie and return its release date."""
    params = {
        'api_key': API_KEY,
        'query': movie_title
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        # Check if results exist
        if data['results']:
            first_result = data['results'][0]
            return {
                'title': first_result['title'],
                'release_date': first_result.get('release_date', 'Unknown'),
                'overview': first_result.get('overview', 'No overview available.')
            }
        else:
            return f"No results found for '{movie_title}'."
    else:
        return f"Error: {response.status_code} - {response.text}"
    
movie_title = "Inception"
movie_info = search_movie(movie_title)
print(movie_info)

{'title': 'Inception', 'release_date': '2010-07-15', 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.'}
